In [2]:
!pip install pyspark


[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install findspark

  Using cached findspark-2.0.1-py2.py3-none-any.whl (4.4 kB)



[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import pyspark
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]") \
                    .appName('microdados_enem') \
                    .getOrCreate()
from pyspark.sql.types import StructType,StructField, StringType, IntegerType

## Fazendo a leitura do arquivo

In [5]:
df = spark.read.options(header='True', inferSchema='True', delimiter=';') \
  .csv("data\\MICRODADOS_ENEM_2020.csv")

#### Fact Enem

Cria a tabela fact_enem

In [6]:
enem = df[["NU_INSCRICAO",
            "NU_NOTA_CN",
            "NU_NOTA_CH",
            "NU_NOTA_LC",
            "NU_NOTA_MT",
            "NU_NOTA_REDACAO",
            "CO_MUNICIPIO_ESC",
            "TP_ESCOLA",
            "TP_SEXO",
            "TP_COR_RACA",
            "TP_PRESENCA_CN",
            "TP_PRESENCA_CH",
            "TP_PRESENCA_LC",
            "TP_PRESENCA_MT",
            "TP_STATUS_REDACAO"]]

In [7]:
enem.createOrReplaceTempView("enem")

#### Dim tp_sexo

Cria a tabela dimensão tp_sexo

In [7]:
data = [("M","masculino"),
        ("F","feminino")]

schema = StructType([ \
    StructField("ID",IntegerType(),True), \
    StructField("SEXO",StringType(),True), \
  ])

# Create dataframe tp_sexo
tp_sexo = spark.createDataFrame(data=data,schema=schema)
tp_sexo.createOrReplaceTempView("tp_sexo")

#### Dim tp_escola

Cria a tabela dimensão tp_escola

In [8]:
data = [(1,"Sem Resposta"),
        (2,"Publica"),
        (3, "Privada"),
        (4, "Exterior")]

schema = StructType([ \
    StructField("ID",IntegerType(),True), \
    StructField("ESCOLA",StringType(),True), \
  ])

# Create dataframe tp_sexo
tp_escola = spark.createDataFrame(data=data,schema=schema)
tp_escola.createOrReplaceTempView("tp_escola")

#### Dim tp_cor_raca

Cria a tabela dimensão tp_cor_raca

In [9]:
data = [(0,"Sem Resposta"),
        (1,"Branca"),
        (2, "Preta"),
        (3, "Parda"),
        (4, "Amarela"),
        (5, "Indigena")]
schema = StructType([ \
    StructField("ID",IntegerType(),True), \
    StructField("COR_RACA",StringType(),True), \
  ])

# Create dataframe tp_sexo
tp_cor_raca = spark.createDataFrame(data=data,schema=schema)
tp_cor_raca.createOrReplaceTempView("tp_cor_raca")

#### Dim tp_presenca_cn

Cria a tabela dimensão tp_presenca_cn

In [10]:
data = [(0,"Faltou"),
        (1,"Presente"),
        (2, "Eliminado")]
schema = StructType([ \
    StructField("ID",IntegerType(),True), \
    StructField("PRESENCA",StringType(),True), \
  ])

# Create dataframe tp_presenca
tp_presenca = spark.createDataFrame(data=data,schema=schema)
tp_presenca.createOrReplaceTempView("tp_presenca_cn")

#### Dim tp_presenca_ch

Cria a tabela dimensão tp_presenca_ch

In [11]:
data = [(0,"Faltou"),
        (1,"Presente"),
        (2, "Eliminado")]
schema = StructType([ \
    StructField("ID",IntegerType(),True), \
    StructField("PRESENCA",StringType(),True), \
  ])

# Create dataframe tp_presenca
tp_presenca = spark.createDataFrame(data=data,schema=schema)
tp_presenca.createOrReplaceTempView("tp_presenca_ch")

#### Dim tp_presenca_lc

Cria a tabela dimensão tp_presenca_lc

In [12]:
data = [(0,"Faltou"),
        (1,"Presente"),
        (2, "Eliminado")]
schema = StructType([ \
    StructField("ID",IntegerType(),True), \
    StructField("PRESENCA",StringType(),True), \
  ])

# Create dataframe tp_presenca
tp_presenca = spark.createDataFrame(data=data,schema=schema)
tp_presenca.createOrReplaceTempView("tp_presenca_lc")

#### Dim tp_presenca_mt

Cria a tabela dimensão tp_presenca_mt

In [13]:
data = [(0,"Faltou"),
        (1,"Presente"),
        (2, "Eliminado")]
schema = StructType([ \
    StructField("ID",IntegerType(),True), \
    StructField("PRESENCA",StringType(),True), \
  ])

# Create dataframe tp_presenca
tp_presenca = spark.createDataFrame(data=data,schema=schema)
tp_presenca.createOrReplaceTempView("tp_presenca_mt")

#### Dim tp_status_redacao

Cria a tabela dimensão tp_status_redacao

In [14]:
data = [(1,"Sem Problemas"),
        (2,"Anulada"),
        (3, "Copia Texto Motivador"),
        (4, "Em Branco"),
        (6, "Fuga ao Tema"),
        (7, "Nao Atendimento Tipo Textual"),
        (8, "Texto Insuficiente"),
        (9, "Parte Desconectada")]
schema = StructType([ \
    StructField("ID",IntegerType(),True), \
    StructField("STATUS",StringType(),True), \
  ])

# Create dataframe tp_presenca
tp_status_redacao = spark.createDataFrame(data=data,schema=schema)
tp_status_redacao.createOrReplaceTempView("tp_status_redacao")

### Querys

Cria uma view

In [15]:
queryEnem = """
select
*
from (
    select
    e.NU_INSCRICAO,
    e.CO_MUNICIPIO_ESC,
    c.COR_RACA,
    x.SEXO,
    s.ESCOLA,
    e.NU_NOTA_CN,
    e.NU_NOTA_CH,
    e.NU_NOTA_LC,
    e.NU_NOTA_MT,
    e.NU_NOTA_REDACAO,
    round((e.NU_NOTA_CN + e.NU_NOTA_CH + e.NU_NOTA_LC + e.NU_NOTA_MT + e.NU_NOTA_REDACAO) / 5,2) as MEDIA
    from enem e
    inner join tp_escola s on e.TP_ESCOLA = s.ID
    inner join tp_status_redacao r on e.TP_STATUS_REDACAO = r.ID
    inner join tp_presenca_cn cn on e.TP_PRESENCA_CN = cn.ID
    inner join tp_presenca_ch ch on e.TP_PRESENCA_CH = ch.ID
    inner join tp_presenca_lc lc on e.TP_PRESENCA_LC = lc.ID
    inner join tp_presenca_mt mt on e.TP_PRESENCA_MT = mt.ID
    inner join tp_sexo x on e.TP_SEXO = x.ID
    inner join tp_cor_raca c on e.TP_COR_RACA = c.ID
    where e.TP_STATUS_REDACAO = 1 and
    e.TP_PRESENCA_CN = 1 and
    e.TP_PRESENCA_CH = 1 and
    e.TP_PRESENCA_LC = 1 and
    e.TP_PRESENCA_MT = 1 and
    s.ID != 1
) e
order by MEDIA desc
"""

In [16]:
vw_enem = spark.sql(queryEnem)

In [17]:
vw_enem.createOrReplaceTempView("vw_enem")

In [18]:
vw_enem.show()

+------------+----------------+--------+---------+-------+----------+----------+----------+----------+---------------+------+
|NU_INSCRICAO|CO_MUNICIPIO_ESC|COR_RACA|     SEXO| ESCOLA|NU_NOTA_CN|NU_NOTA_CH|NU_NOTA_LC|NU_NOTA_MT|NU_NOTA_REDACAO| MEDIA|
+------------+----------------+--------+---------+-------+----------+----------+----------+----------+---------------+------+
|200001357436|         3304557|  Branca|masculino|Privada|     795.8|     839.5|     713.5|     975.0|            940|852.76|
|200001850224|         3550308|  Branca| feminino|Privada|     800.8|     862.6|     716.9|     896.4|            980|851.34|
|200006051511|         2507507|  Branca| feminino|Privada|     756.7|     819.2|     725.4|     952.4|           1000|850.74|
|200005145247|         3530607| Amarela|masculino|Privada|     754.0|     838.0|     687.0|     975.0|            980| 846.8|
|200001423299|         3106200|  Branca|masculino|Privada|     798.3|     742.7|     759.6|     964.4|            960|

#### Qual a escola com a maior média de notas?

In [19]:
# Qual a escola com a maior média de notas?
queryEscolaMaiorMedia = """
    select
    CO_MUNICIPIO_ESC as ESCOLA,
    ESCOLA as TIPO,
    avg(MEDIA) as MEDIA
    from vw_enem
    group by 1, 2
    order by 3 desc
    limit 5
"""
escola_maior_media = spark.sql(queryEscolaMaiorMedia)
escola_maior_media.show()

+-------+-------+-------+
| ESCOLA|   TIPO|  MEDIA|
+-------+-------+-------+
|3113701|Privada| 750.46|
|3108909|Privada|741.615|
|3550209|Privada| 731.36|
|4121406|Privada|729.105|
|3158904|Publica| 706.56|
+-------+-------+-------+



#### Qual o aluno com a maior média de notas e o valor dessa média?

In [20]:
# Qual o aluno com a maior média de notas e o valor dessa média?
queryAlunoMaiorMedia = """
    select
    NU_INSCRICAO,
    SEXO,
    COR_RACA,
    MEDIA
    from vw_enem
    order by 4 desc
    limit 5
"""
aluno_maior_media = spark.sql(queryAlunoMaiorMedia)
aluno_maior_media.show()

+------------+---------+--------+------+
|NU_INSCRICAO|     SEXO|COR_RACA| MEDIA|
+------------+---------+--------+------+
|200001357436|masculino|  Branca|852.76|
|200001850224| feminino|  Branca|851.34|
|200006051511| feminino|  Branca|850.74|
|200005145247|masculino| Amarela| 846.8|
|200001423299|masculino|  Branca| 845.0|
+------------+---------+--------+------+



#### Qual a média geral?

In [21]:
# Qual a média geral?
queryMediaGeral = """
    select
    round(avg(MEDIA),2) as MEDIA_GERAL
    from vw_enem
"""
media_geral = spark.sql(queryMediaGeral)
media_geral.show()

+-----------+
|MEDIA_GERAL|
+-----------+
|     532.41|
+-----------+



#### Qual a média por disciplina?

In [22]:
# Qual a média por disciplina?
queryMediaDisciplina = """
    select
    round(avg(NU_NOTA_CN),2) as MEDIA_CN,
    round(avg(NU_NOTA_CH),2) as MEDIA_CH,
    round(avg(NU_NOTA_LC),2) as MEDIA_LC,
    round(avg(NU_NOTA_MT),2) as MEDIA_MT,
    round(avg(NU_NOTA_REDACAO),2) as REDACAO
    from vw_enem
"""
media_disciplina = spark.sql(queryMediaDisciplina)
media_disciplina.show()

+--------+--------+--------+--------+-------+
|MEDIA_CN|MEDIA_CH|MEDIA_LC|MEDIA_MT|REDACAO|
+--------+--------+--------+--------+-------+
|  489.99|  514.37|  527.64|  524.01| 606.05|
+--------+--------+--------+--------+-------+



#### Qual a média por Sexo?

In [23]:
# Qual a média por Sexo?
queryMediaSexo = """
    select
    SEXO,
    round(avg(MEDIA),2) as MEDIA
    from vw_enem
    group by 1
    order by 2
"""
media_sexo = spark.sql(queryMediaSexo)
media_sexo.show()

+---------+------+
|     SEXO| MEDIA|
+---------+------+
| feminino| 526.2|
|masculino|541.55|
+---------+------+



#### Qual a média por Etnia?

In [24]:
# Qual a média por Etnia?
queryMediaEtnia = """
    select
    COR_RACA,
    round(avg(MEDIA),2) as MEDIA
    from vw_enem
    group by 1
    order by 2
"""
media_etnia = spark.sql(queryMediaEtnia)
media_etnia.show()

+------------+------+
|    COR_RACA| MEDIA|
+------------+------+
|    Indigena|479.92|
|       Preta|505.55|
|       Parda|512.17|
|Sem Resposta|526.45|
|     Amarela|529.35|
|      Branca| 560.3|
+------------+------+



#### Qual o número total de Inscritos?

In [25]:
# Qual o número total de Inscritos?
queryTotalInscritos = """
    select
    count(NU_INSCRICAO) as TOTAL_INSCRITOS
    from enem
"""
total_inscritos = spark.sql(queryTotalInscritos)
total_inscritos.show()

+---------------+
|TOTAL_INSCRITOS|
+---------------+
|        5783109|
+---------------+



#### Qual o % de Ausentes?

In [26]:
# Qual o % de Ausentes?
queryPercentAusentes = """
    select
    round(((AUSENTES_CN * 100) / TOTAL),2) as PERCENT_AUSENTES_CN,
    round(((AUSENTES_CH * 100) / TOTAL),2) as PERCENT_AUSENTES_CH
    from (
        select
        count(NU_INSCRICAO) as TOTAL,
        sum(case when TP_PRESENCA_CN = 0 then 1 else 0 end) as AUSENTES_CN,
        sum(case when TP_PRESENCA_CH = 0 then 1 else 0 end) as AUSENTES_CH,
        sum(case when TP_PRESENCA_LC = 0 then 1 else 0 end) as AUSENTES_LC,
        sum(case when TP_PRESENCA_MT = 0 then 1 else 0 end) as AUSENTES_MT
        from enem
    ) t
"""
ausentes = spark.sql(queryPercentAusentes)
ausentes.show()

+-------------------+-------------------+
|PERCENT_AUSENTES_CN|PERCENT_AUSENTES_CH|
+-------------------+-------------------+
|              55.06|               52.3|
+-------------------+-------------------+

